In [1]:
import json

# Define your intents
intents ={
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hello", "Hi", "Hey", "Good morning", "Good evening"],
            "responses": ["Hello! How can I assist you with your shopping today?", "Hi there! Need help finding something?", "Hey! How can I help?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you", "Goodbye", "Take care"],
            "responses": ["Goodbye! Happy shopping!", "See you soon!", "Take care and visit us again!"]
        },
        {
            "tag": "order_tracking",
            "patterns": ["Where is my order?", "Track my order", "Order status", "Can you check my order?"],
            "responses": ["You can track your order using the tracking link sent to your email.", "Please visit your account's order history for real-time tracking.", "Enter your order number on our tracking page to see the status."]
        },
        {
            "tag": "payment_methods",
            "patterns": ["What payment methods do you accept?", "Can I pay with PayPal?", "Do you accept credit cards?"],
            "responses": ["We accept credit/debit cards, PayPal, and UPI payments.", "Yes, we accept Visa, MasterCard, PayPal, and other digital wallets."]
        },
        {
            "tag": "return_policy",
            "patterns": ["What is your return policy?", "Can I return a product?", "How do I request a return?"],
            "responses": ["You can return a product within 30 days if it's in original condition.", "To initiate a return, go to your orders and select 'Request a Return'."]
        },
        {
            "tag": "refund_time",
            "patterns": ["When will I get my refund?", "How long does a refund take?"],
            "responses": ["Refunds are processed within 5-7 business days after approval.", "Once your return is received and inspected, refunds are issued within a week."]
        },
        {
            "tag": "shipping_info",
            "patterns": ["How long does delivery take?", "When will my order arrive?", "Shipping time"],
            "responses": ["Standard delivery takes 3-5 business days.", "Expedited shipping options are available at checkout for faster delivery."]
        },
        {
            "tag": "product_availability",
            "patterns": ["Is this product in stock?", "Do you have this item available?", "When will this product be restocked?"],
            "responses": ["You can check product availability on the product page.", "If an item is out of stock, you can sign up for restock notifications."]
        }
    ]
}


# Save as a JSON file
with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)

print("intents.json file created successfully!")


intents.json file created successfully!


In [ ]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab') # Download the punkt_tab resource

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents file
with open("intents.json") as file:
    intents = json.load(file)

# ... (rest of your code)

# Load intents file
with open("intents.json") as file:
    intents = json.load(file)

# Preprocessing Data
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in document[0]]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Build Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train Model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print("Model training completed!")

# Load Model & Define Chat Function
model = load_model('chatbot_model.h5')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{'intent': classes[r[0]], 'probability': str(r[1])} for r in results]

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "I'm sorry, I don't understand."

print("Chatbot is running! Type 'quit' to exit.")
while True:
    message = input("You: ")
    if message.lower() in ['quit', 'exit', 'bye']:
        print("Goodbye! Have a great day!")
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Bot:", res)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0707 - loss: 2.1766  
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0761 - loss: 2.1337      
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1743 - loss: 2.0768    
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1468 - loss: 2.0546     
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2121 - loss: 1.9386 
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3624 - loss: 1.8823 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3426 - loss: 1.8316 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4150 - loss: 1.7644 
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6136 - loss: 1.5921 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3849 - loss: 1.7439 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4711 - loss: 1.6337 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5922 - 

Model training completed!


Chatbot is running! Type 'quit' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Bot: Hey! How can I help?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Bot: You can track your order using the tracking link sent to your email.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Bot: We accept credit/debit cards, PayPal, and UPI payments.
